# Purpose
This notebook cleans old intake.catalyst.coop logs that were downloaded from Cloud Logging Explorer that were not captured in the `intake-bucket-logs` sink. This notebook shouldn't be reused but could be helpful if we forget to create a log sink for a bucket we care about. 

In [7]:
import json
from pathlib import Path

log_path = Path("downloaded-logs-20220707-173840.json").resolve()

f = open(log_path)
logs = json.load(f)

In [3]:
log = logs[0]
print(log.keys())
print(log["protoPayload"].keys())
print(log["protoPayload"]["requestMetadata"]["requestAttributes"])

dict_keys(['protoPayload', 'insertId', 'resource', 'timestamp', 'severity', 'logName', 'receiveTimestamp'])
dict_keys(['@type', 'status', 'authenticationInfo', 'requestMetadata', 'serviceName', 'methodName', 'authorizationInfo', 'resourceName', 'serviceData', 'request', 'metadata', 'resourceLocation'])
{'time': '2022-04-18T23:53:45.290293436Z', 'auth': {}}


In [4]:
for record in logs:
    # BQ doesn't allow special characters in field names
    record["protoPayload"].pop("@type", None)
    
    # Columns that exist in the export but not in the fields generated by the log sink
    record["protoPayload"].pop("serviceData", None)
    record["protoPayload"].pop("request", None)
    record["protoPayload"].pop("metadata", None)
    
    # BQ Doesn't handle empty records well
    record["protoPayload"]["requestMetadata"].pop("destinationAttributes", None)
    record["protoPayload"]["requestMetadata"]["requestAttributes"].pop("auth", None)
    for info in record["protoPayload"]["authorizationInfo"]:
        info.pop("resourceAttributes", None)
        
    # Log sink and export column rename
    record["protopayload_auditlog"] = record.pop("protoPayload")
    
    

In [5]:
with open("clean_old_intake_logs.json", "w") as outfile:
    json.dump(logs, outfile)

In [6]:
# Convert to new line delimeted json
!cat clean_old_intake_logs.json | jq -c '.[]' > nld_clean_old_intake_logs.json